In [1]:
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime, Float
import psycopg2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import geopy
from geopy.geocoders import Nominatim
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# POSTGRES LOCAL CONNECTION

In [3]:
# # link to postgresql on my local machine
# # https://naysan.ca/2020/05/31/postgresql-to-pandas/
# param_dic = {
#     "host"      : "localhost",  
#     "database"  : "Meteorites",
#     "user"      : "postgres",
#     "password"  : "postgres"
# }
# def connect(params_dic):
#     """ Connect to the PostgreSQL database server """
#     conn = None
#     try:
#         # connect to the PostgreSQL server
#         print('Connecting to the PostgreSQL database...')
#         conn = psycopg2.connect(**params_dic)
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         sys.exit(1) 
#     print("Connection successful")
#     return conn

# AWS DB CONNECTION

In [4]:
# link to aws
# https://naysan.ca/2020/05/31/postgresql-to-pandas/
param_dic = {
    "host"      : "meteorite-ml-db.cl3jquwwq7kn.us-east-1.rds.amazonaws.com",  
    "database"  : "meteorite_ml_db",
    "user"      : "postgres",
    "password"  : "project4team5"
}
def connect(params_dic):
    """ Connect to the PostgreSQL/AWS database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL/AWS database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [5]:
# link to aws
# https://naysan.ca/2020/05/31/postgresql-to-pandas/
param_dic = {
    "host"      : "localhost",  
    "database"  : "Meteorites",
    "user"      : "postgres",
    "password"  : "postgres"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [6]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [7]:
landcover_cols=["id","state_abbrev","variable","value"]
state_cols=["state_abbrev","state","FIPS","area_sqkm","country"]
meteorite_type_cols=["recclass","meteorite_class_subclass","meteorite_class","meteorite_type"]
meteorite_main_cols=["id","name","recclass","mass_grams","fall","year","reclat","reclong","geolocation","geometry","elevation","country","state_abbrev"]

# dashboard5_cols = ["name","recclass","meteorite_class_subclass","meteorite_class","meteorite_type","mass_grams","fall","year","reclat","reclong","geolocation","geometry","elevation","country","state_abbrev","state","FIPS","area_sqkm","country"]

# select statement mm.name, mm.recclass,mt.meteorite_class_subclass, mt.meteorite_class,mt.meteorite_type,mm.mass_grams,mm.fall, mm.year,mm.reclat,mm.reclong,mm.geolocation,mm.geometry,mm.elevation,mm.country,mm.state_abbrev, s.state, s.FIPS, s.area_sqkm, s.country]

In [8]:
conn = connect(param_dic)


Connecting to the PostgreSQL database...
Connection successful


In [9]:
state_df = postgresql_to_dataframe(conn,"select * from state",state_cols)
state_df.head()

,state_abbrev,state,FIPS,area_sqkm,country
0,AL,Alabama,1,135767,United States
1,AK,Alaska,2,1723337,United States
2,AZ,Arizona,4,295234,United States
3,AR,Arkansas,5,137732,United States
4,CA,California,6,423967,United States


In [10]:
landcover_df = postgresql_to_dataframe(conn,"select * from landcover",landcover_cols)
landcover_df.head()

,id,state_abbrev,variable,value
0,1,AL,FOREST,64.872148
1,2,AL,GRSL,19.904490
2,3,AL,WETL,0.176988
3,4,AL,SHRUBL,1.222573
4,5,AL,SPARSE_VEGETATION,0.034765


In [11]:
# remove ID column and set state abbreviation to index
landcover_df.drop(['id'], axis=1, inplace=True)
landcover_df.set_index('state_abbrev', inplace=True)
landcover_df

,variable,value
state_abbrev,,
AL,FOREST,64.872148
AL,GRSL,19.904490
AL,WETL,0.176988
AL,SHRUBL,1.222573
AL,SPARSE_VEGETATION,0.034765
...,...,...
VA,SPARSE_VEGETATION,0.025975
VA,CROPL,10.927344
VA,URBAN,2.967337


In [12]:
# flatten out landcover data
# pivot table to create columns for each variable
landcovery = (landcover_df.pivot_table(index = landcover_df.index, columns='variable',values = 'value', aggfunc=lambda x: x, fill_value=0)
       .rename_axis(None, axis='columns')
      
)
landcovery.reset_index(inplace=True)
print(len(landcovery))
landcovery.head(2)

51


,state_abbrev,BARE,CROPL,FOREST,GRSL,SHRUBL,SPARSE_VEGETATION,URBAN,WATER,WETL
0,AK,10.875705,0.045602,34.319849,10.628187,20.993753,15.991683,0.014371,5.340909,1.789941
1,AL,0.189766,10.177265,64.872148,19.904490,1.222573,0.034765,1.659720,1.762286,0.176988


In [13]:
meteorite_type_df = postgresql_to_dataframe(conn,"select * from meteorite_type",meteorite_type_cols)
meteorite_type_df.head()

,recclass,meteorite_class_subclass,meteorite_class,meteorite_type
0,H5,Chrondrite - ordinary,Chrondrite,Chrondrite
1,L6,Chrondrite - ordinary,Chrondrite,Chrondrite
2,L5,Chrondrite - ordinary,Chrondrite,Chrondrite
3,"Iron, ungrouped",Iron - other,Iron,Iron
4,"Iron, IVA",Iron - magmatic,Iron,Iron


In [14]:
meteorite_main_df = postgresql_to_dataframe(conn,"select * from meteorite_main",meteorite_main_cols)
meteorite_main_df.head()

,id,name,recclass,mass_grams,fall,year,reclat,reclong,geolocation,geometry,elevation,country,state_abbrev
0,2276,Allegan,H5,32000.0,Fell,1899,42.53333,-85.88333,"(42.53333, -85.88333)",POINT (-85.88333 42.53333),225.0,United States,MI
1,5401,Coleman,L6,469.0,Fell,1994,43.76111,-84.50778,"(43.76111, -84.50778)",POINT (-84.50778 43.76111),218.0,United States,MI
2,22766,Rose City,H5,10600.0,Fell,1921,44.51667,-83.95000,"(44.51667, -83.95)",POINT (-83.95 44.51667),387.0,United States,MI
3,24337,Worden,L5,1551.0,Fell,1997,42.38467,-83.61150,"(42.38467, -83.6115)",POINT (-83.6115 42.38467),279.0,United States,MI
4,10955,Grand Rapids,"Iron, ungrouped",51700.0,Found,1883,42.96667,-85.76667,"(42.96667, -85.76667)",POINT (-85.76667 42.96667),225.0,United States,MI


In [15]:
dashboard5_cols = ["name","recclass","meteorite_class_subclass","meteorite_class","meteorite_type","mass_grams","fall","year","reclat","reclong","geolocation","geometry","elevation","country","state_abbrev","state","FIPS","area_sqkm","country"]

# select statement mm.name, mm.recclass,mt.meteorite_class_subclass, mt.meteorite_class,mt.meteorite_type,mm.mass_grams,mm.fall, mm.year,mm.reclat,mm.reclong,mm.geolocation,mm.geometry,mm.elevation,mm.country,mm.state_abbrev, s.state, s.FIPS, s.area_sqkm, s.country]

In [16]:
# pull in all data except for landcover using sql
dashboard5_df = postgresql_to_dataframe(conn,"select mm.name, mm.recclass,mt.meteorite_class_subclass, mt.meteorite_class,mt.meteorite_type,mm.mass_grams,mm.fall, mm.year,mm.reclat,mm.reclong,mm.geolocation,mm.geometry,mm.elevation,mm.country,mm.state_abbrev, s.state, s.FIPS, s.area_sqkm, s.country from meteorite_main mm, meteorite_type mt, state s where mm.state_abbrev = s.state_abbrev and mm.recclass = mt.recclass;",dashboard5_cols) 
dashboard5_df.head()

,name,recclass,meteorite_class_subclass,meteorite_class,meteorite_type,mass_grams,fall,year,reclat,reclong,geolocation,geometry,elevation,country,state_abbrev,state,FIPS,area_sqkm,country
0,Allegan,H5,Chrondrite - ordinary,Chrondrite,Chrondrite,32000.0,Fell,1899,42.53333,-85.88333,"(42.53333, -85.88333)",POINT (-85.88333 42.53333),225.0,United States,MI,Michigan,26,250487,United States
1,Coleman,L6,Chrondrite - ordinary,Chrondrite,Chrondrite,469.0,Fell,1994,43.76111,-84.50778,"(43.76111, -84.50778)",POINT (-84.50778 43.76111),218.0,United States,MI,Michigan,26,250487,United States
2,Rose City,H5,Chrondrite - ordinary,Chrondrite,Chrondrite,10600.0,Fell,1921,44.51667,-83.95000,"(44.51667, -83.95)",POINT (-83.95 44.51667),387.0,United States,MI,Michigan,26,250487,United States
3,Worden,L5,Chrondrite - ordinary,Chrondrite,Chrondrite,1551.0,Fell,1997,42.38467,-83.61150,"(42.38467, -83.6115)",POINT (-83.6115 42.38467),279.0,United States,MI,Michigan,26,250487,United States
4,Grand Rapids,"Iron, ungrouped",Iron - other,Iron,Iron,51700.0,Found,1883,42.96667,-85.76667,"(42.96667, -85.76667)",POINT (-85.76667 42.96667),225.0,United States,MI,Michigan,26,250487,United States


In [18]:
print(len(dashboard5_df))

1648


In [17]:
conn.close()

In [19]:
dashboard4_df = pd.merge(dashboard5_df,landcovery, on = 'state_abbrev')
print(len(dashboard4_df))
dashboard4_df.head()

1648


,name,recclass,meteorite_class_subclass,meteorite_class,meteorite_type,mass_grams,fall,year,reclat,reclong,...,country,BARE,CROPL,FOREST,GRSL,SHRUBL,SPARSE_VEGETATION,URBAN,WATER,WETL
0,Allegan,H5,Chrondrite - ordinary,Chrondrite,Chrondrite,32000.0,Fell,1899,42.53333,-85.88333,...,United States,0.241572,35.820019,52.486729,4.448891,0.668927,0.026182,3.350872,2.644462,0.312346
1,Coleman,L6,Chrondrite - ordinary,Chrondrite,Chrondrite,469.0,Fell,1994,43.76111,-84.50778,...,United States,0.241572,35.820019,52.486729,4.448891,0.668927,0.026182,3.350872,2.644462,0.312346
2,Rose City,H5,Chrondrite - ordinary,Chrondrite,Chrondrite,10600.0,Fell,1921,44.51667,-83.95000,...,United States,0.241572,35.820019,52.486729,4.448891,0.668927,0.026182,3.350872,2.644462,0.312346
3,Worden,L5,Chrondrite - ordinary,Chrondrite,Chrondrite,1551.0,Fell,1997,42.38467,-83.61150,...,United States,0.241572,35.820019,52.486729,4.448891,0.668927,0.026182,3.350872,2.644462,0.312346
4,Grand Rapids,"Iron, ungrouped",Iron - other,Iron,Iron,51700.0,Found,1883,42.96667,-85.76667,...,United States,0.241572,35.820019,52.486729,4.448891,0.668927,0.026182,3.350872,2.644462,0.312346


In [20]:
# write US meteorite data into csv file
dashboard4_df.to_csv('Resources/machine_learning/meteorite_dashboard_data.csv')

In [ ]:
# logR_accurate_counts.plot.bar(rot = 0, title = 'Were Logistic Regression predictions accurate?')

In [ ]:
# # Create a scatter plot with two of the features
# plt.scatter(df2['mass_grams'], df2['elevation'], c=labels)
# plt.show()